<a href="https://colab.research.google.com/github/ashikshafi08/Learning-Fastai/blob/main/NLP_Deep_Dive_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
!nvidia-smi

Wed Jan 25 02:53:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    31W /  70W |  12670MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [32]:
import fastai 
from fastai.vision.all import * 


In [33]:
!pip install sentencepiece!=0.1.90,!=0.1.91

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Language model -> a model that has been trained to guess the next word in a text. This is a self supervised learning, hence we dont give labels to our model just feed it lots of texts. 

**Self Supervised Learning**
Training a model using labels that are embedded in the independent variable (X) rather than requiring a external labels. For instance, training a model to predict the next word. 


Fine tune our language model for IMDB rather pretraining it with the Wikipedia texts. 

Fine tune the language model prior to fine-tuning the classification model. For instance, IMDb sentiment analysis task, the dataset includes 50,000 additional movie reviews that dont have any labels to them. We can use these reviews to fine tune the pretrained language model, which trained only on Wikipedia articles this will help in predicting the next word for.
Fine tune the language model prior to fine-tuning the classification model. For instance, IMDb sentiment analysis task, the dataset includes 50,000 additional movie reviews that dont have any labels to them. We can use these reviews to fine tune the pretrained language model, which trained only on Wikipedia articles this will help in predicting the next word of a movie review. 


#### Text Processing 

First we concatentate all of the documents in our dataset into one big long strings and split it into words (or tokens), this is gives us the long list of words. 

Vocab will contain mix of common words that are in the vocbulary of the pretrained model we have and also new words specific to our corpus. 

- Our embedding matrix will be built accordingly for words that are in the vocabulary of our pretrained model. 
- For the new words we initialize a random vector. 


The steps are as follows: 
1. Tokenization 
  - Convert text into a list of words (substring, characters etc.) 
2. Numericalizaion
  - Convert the vocab (list of all unique words) into number by looking up its index in the vocab. 
3. Language model data loader creation 
  - In here `LMDataLoader` class that automatically handles creating a dependent varibale (y) that is offset from the independent variable (x). 
  - It also does the shuffle of the training data in a way where X and y maintain their structure. 
4. Language model creation
  - building from scratch -> RNN, for now a deep neural network. 

### Tokenization 

How do we need how to split the words? The punctuation, other chemical corpus, hyphenated words etc. Even languages like Japenese and Chinese that dont uses the convention english words. 

There are three tokenization methods: 
1. Word-based -> split on spaces, general punctuation marks.
2. Subword based -> into smaller parts 
3. Character based -> split sentence into its individual characters. 


**Token** -> one element of a list created by the tokenization process. 




#### Word Tokenization with fastai 

In [34]:
from fastai.text.all import * 
path = untar_data(URLs.IMDB)

path.ls()

(#7) [Path('/root/.fastai/data/imdb/unsup'),Path('/root/.fastai/data/imdb/imdb.vocab'),Path('/root/.fastai/data/imdb/train'),Path('/root/.fastai/data/imdb/tmp_lm'),Path('/root/.fastai/data/imdb/test'),Path('/root/.fastai/data/imdb/README'),Path('/root/.fastai/data/imdb/tmp_clas')]

In [35]:
path.ls()[:5]

(#5) [Path('/root/.fastai/data/imdb/unsup'),Path('/root/.fastai/data/imdb/imdb.vocab'),Path('/root/.fastai/data/imdb/train'),Path('/root/.fastai/data/imdb/tmp_lm'),Path('/root/.fastai/data/imdb/test')]

In [36]:
# Getting the text files 
files = get_text_files(path , folders = ['train', 'test', 'unsup'])

In [37]:
(path / "unsup").ls()

(#50000) [Path('/root/.fastai/data/imdb/unsup/2087_0.txt'),Path('/root/.fastai/data/imdb/unsup/31436_0.txt'),Path('/root/.fastai/data/imdb/unsup/15120_0.txt'),Path('/root/.fastai/data/imdb/unsup/43451_0.txt'),Path('/root/.fastai/data/imdb/unsup/7427_0.txt'),Path('/root/.fastai/data/imdb/unsup/49120_0.txt'),Path('/root/.fastai/data/imdb/unsup/8469_0.txt'),Path('/root/.fastai/data/imdb/unsup/4359_0.txt'),Path('/root/.fastai/data/imdb/unsup/7972_0.txt'),Path('/root/.fastai/data/imdb/unsup/132_0.txt')...]

In [38]:
files[0].open().read()

'Seven years ago, when I was in high school, I was assigned to read and study John Steinbeck\'s 1937 novella, "Of Mice and Men", in my English class, and it\'s no surprise that I still remember it clearly now, as it\'s not a story to forget. Shortly after reading the book, we were shown the 1992 film adaptation in that class. I first heard about this much earlier adaptation of the book around that time, but it took me seven years to finally get around to watching it, even though I had been intending to watch it for quite a while by the time I did. I knew this version was widely praised, which gave me fairly high expectations. It may not have fully met these expectations, but if not, at least it came very close, and I would say it\'s just as memorable as the book.<br /><br />In the 1930\'s Great Depression era, George Milton and Lennie Small are two migrant ranch workers in California who dream of having their own place someday. Lennie is big and strong, but he is also mentally handicap

In [39]:
# Example text 

txt = files[0].open().read()
txt[:40]

'Seven years ago, when I was in high scho'

Rather using the spacy's tokenizer `SpaceTokenizer` we will use the fastai's `WordTokenizer`. 

`coll_repr(collection , n)` -> function to display the results. n items of the collection. 

In [40]:
# Spacy's Tokenizer 
spacy = WordTokenizer()


# Tokens -> convert words into tokens
# [txt] -> strings into a list of strings
toks = first(spacy([txt]))

# Display the result 
print(coll_repr(toks , 30))

(#773) ['Seven','years','ago',',','when','I','was','in','high','school',',','I','was','assigned','to','read','and','study','John','Steinbeck',"'s",'1937','novella',',','"','Of','Mice','and','Men','"'...]


It also splits the term `I'd -> I 'd` as a seperate token which is true. Spacy handles this for us. 

In [41]:
# Fastai's tokenizer 

tkn = Tokenizer(spacy) # Takes in the tokenizer
print(coll_repr(tkn(txt) , 31))

(#841) ['xxbos','xxmaj','seven','years','ago',',','when','i','was','in','high','school',',','i','was','assigned','to','read','and','study','xxmaj','john','xxmaj','steinbeck',"'s",'1937','novella',',','"','of','xxmaj'...]


The tokens that start with the characters `xx` are special tokens. 

We are simplifying the english language to a simplified tokenized language for the model to learn better. 

- Capitalized word will be replaced with a special capitilization token, followed by the lowercase version of the word. XXCAP token -> lowercase word of the captial. 

By this tokens we can save the compute and the memory resources, and these tokens determine whether its capital or not. 


- xxbos -> beginning of stream, start of a new text. 
- xxmaj -> indicates next word begin with a capital. 
- xxunk -> indicates next word in unknown


In [42]:
# The rules used 
defaults.text_proc_rules

# Read page 335 for more info on these 

[<function fastai.text.core.fix_html(x)>,
 <function fastai.text.core.replace_rep(t)>,
 <function fastai.text.core.replace_wrep(t)>,
 <function fastai.text.core.spec_add_spaces(t)>,
 <function fastai.text.core.rm_useless_spaces(t)>,
 <function fastai.text.core.replace_all_caps(t)>,
 <function fastai.text.core.replace_maj(t)>,
 <function fastai.text.core.lowercase(t, add_bos=True, add_eos=False)>]

#### Subword Tokenization 

For chinese and japanese letters dont use spaces, and not well defined into the category word as per the english corpus. So we use subwords to tackle these problems. 

We handle this: 
- Analyze the corpus of documents to find the most commonly occuring groups of letters. These becomes the vocab. 
- Tokenize the corpus using this vocab of subword units. 



In [43]:
# Iterating to the corpus and storing them into a list

txts = L(o.open().read() for o in files[:2000])

Now we need to find the common sequences of characters in these excerpts to create a vocab. 

We can use the `setup` special fastai method for our data processing pipelines. 

In [44]:
def subword(sz):
  sp = SubwordTokenizer(vocab_sz=sz)
  sp.setup(txts)
  return " ".join(first(sp([txt]))[:40])

In [45]:
subword(200)

'▁S e ve n ▁ y e ar s ▁a g o , ▁w h en ▁I ▁was ▁in ▁h i g h ▁ s ch o o l , ▁I ▁was ▁a s s i g n ed ▁to'

In [46]:
subword(1000)

'▁Se ven ▁years ▁a go , ▁when ▁I ▁was ▁in ▁high ▁school , ▁I ▁was ▁as s ign ed ▁to ▁read ▁and ▁st u dy ▁John ▁St e in be ck \' s ▁19 3 7 ▁novel la , ▁"'

If we use a smaller vocab size, each token will represent fewer characters and takes more tokens to represent a sentence. 

`_` -> represent the special character 

**Larger vocab** -> fewer tokens per sentence 

  --> fastest training, less memory and less state for the model 
  --> downside : larger embedding matrices, require more data to learn 

Subword tokenization -> scale between character tokenization and word tokenization. 

Importantly it can handle every human language without any specific algorithm.

#### Numericalization with fastai 

Mapping tokens to integers, identical to that of creating a Category variable like in MNIST. 

1. Make a list of all possible levels of that categorical variable (we call this vocab). 
- Replace each level with its index in the vocab. 


tokenization is done in parallel by fastai. 


This is how it works: 
- Have a vocab 
- Fit the vocab to a `Numericalize` and create an object
- Use the numericalize object to convert tokens into numbers. 

In [47]:
# Samples tokens for the demo 

# Map function to use the tokenizer with the texts we need.
toks200 = txts[:200].map(tkn)
toks200[0]

(#841) ['xxbos','xxmaj','seven','years','ago',',','when','i','was','in'...]

In [48]:
# Numericalize 
num = Numericalize() 

# Fitting / or setup in fastai 
num.setup(toks200)

# Displaying
coll_repr(num.vocab , 20)

"(#2128) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the','.',',','and','a','of','to','is','i','it','in'...]"

Two parameters of the `Numericalize()`
- max_vocab : instead of adding all the words, it restricts and replacing all words other than the most common 60,000 with some special unknown token word **`xxunk`**. 

- min_freq : number of times a word can reappear 

The `max_vocab` is used to avoid having a large embedding matrix, so it wont slow down the training and dont end up using too much memory. 

`vocab` -> pass in the custom vocab variable we have. 

In [49]:
nums = num(toks)[:30]
nums

TensorText([  0, 176, 993,  11,  82,   0,  25,  19, 286, 242,  11,   0,  25,
              0,  15, 458,  12,   0,   0,   0,  23,   0,   0,  11,  22,   0,
              0,  12,   0,  22])

We can check by mapping them back to the original text 

In [50]:
" ".join(num.vocab[o] for o in nums)

'xxunk years ago , when xxunk was in high school , xxunk was xxunk to read and xxunk xxunk xxunk \'s xxunk xxunk , " xxunk xxunk and xxunk "'

### Putting our text into Batches for a Language Model 

We can't resize text and we want our language model to read the text in order so that they can predict what the next word is. 

| **So this means each new batch should begin precisely where the previous one left off** |


So we need to divide this array more finely into subarrays of a fixed sequence length, we need to obtain the order within and across these subarrays.

We shuffle order of the documents not the order of the words insid. 

For instance, if we have 50000 tokens and a batch size of 10. Then we will preserve the order of the tokens (1 - 5000 for the first mini-stream / mini-batch, then 5001 to 10000 and so on...) 

With the special tokens xxbos the model knows which one is the beginning of the excerpt or the document. 




In [51]:
# Numericalize the tokens 
nums200 = toks200.map(num)

# Pass into the LMDataLoader 
dl = LMDataLoader(nums200)

dl

In [52]:
# Looking into the dataloader 
x , y = first(dl)
x.shape,  y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [53]:
doc(LMDataLoader) # Default seq len is 72 and bs is 64 

LMDataLoader(dataset, lens=None, cache=2, bs=64, seq_len=72, num_workers=0, *, shuffle: 'bool' = False, verbose: 'bool' = False, do_setup: 'bool' = True, pin_memory=False, timeout=0, batch_size=None, drop_last=False, indexed=None, n=None, device=None, persistent_workers=False, pin_memory_device='', wif=None, before_iter=None, after_item=None, before_batch=None, after_batch=None, after_iter=None, create_batches=None, create_item=None, create_batch=None, retain=None, get_idxs=None, sample=None, shuffle_fn=None, do_batch=None)
A `DataLoader` suitable for language modeling

To get a prettier result with hyperlinks to source code and documentation, install nbdev: pip install nbdev


In [54]:
# Looking into the X 
" ".join(num.vocab[o] for o in x[1][:20])

"are xxunk with you it just gets worse . xxmaj about half the film xxunk xxmaj ash 's struggle against"

In [55]:
# Looking into the y 
" ".join(num.vocab[o] for o in y[1][:20])

"xxunk with you it just gets worse . xxmaj about half the film xxunk xxmaj ash 's struggle against xxunk"

#### Training a Text Classifier 

Two steos to train the sota text classifier using transfer learning" 
- fine tune our language model pretrained on Wikipedia to the corpus of IMDB review. 
- then use the above to train the classifier. 


TextBlock -> tokenization and numericalization happens automatically when `TextBlock` is passed to the `DataBlock`. 



In [56]:
get_imdb = partial(get_text_files , folders = ["train", "test" , "unsup"])
get_imdb

functools.partial(<function get_text_files at 0x7ff64193a8b0>, folders=['train', 'test', 'unsup'])

In [57]:
# Building a Text DataBlock 
dblock_lm = DataBlock(
    blocks = TextBlock.from_folder(path , is_lm = True), 
    get_items = get_imdb , 
    splitter = RandomSplitter(0.1)
)

In [58]:
# DataLoaders 
dls_lm = dblock_lm.dataloaders(path , path = path , bs = 128 , seq_len = 80)

TextBlock.from_folder -> classmethods 

https://stackoverflow.com/questions/12179271/meaning-of-classmethod-and-staticmethod-for-beginner

In [59]:
# Showing the results 
dls_lm.show_batch(max_n = 3)

,text,text_
0,"xxbos xxmaj but certainly a serious contender for one of the worst 10 of all time . \n\n i got this xxup dvd cheap , with xxmaj sandra xxmaj bullock as headliner on the case . xxmaj this is false advertising - she 's on - screen for almost 10 minutes of the movie . \n\n xxmaj on the other hand , there was no other selling point for this movie - the dialog was horrible , the editing was","xxmaj but certainly a serious contender for one of the worst 10 of all time . \n\n i got this xxup dvd cheap , with xxmaj sandra xxmaj bullock as headliner on the case . xxmaj this is false advertising - she 's on - screen for almost 10 minutes of the movie . \n\n xxmaj on the other hand , there was no other selling point for this movie - the dialog was horrible , the editing was apparently"
1,"so much depth . xxmaj the xxmaj doctor seems more vulnerable in this series , maybe because he is the last xxunk . xxmaj the range of emotions which xxmaj chris and xxmaj david show are truly remarkable , and i felt their pain , anger , and sense of adventure right along with them . \n\n xxmaj billie xxmaj piper ( rose ) brings a very human element to each episode , although she is much braver than i","much depth . xxmaj the xxmaj doctor seems more vulnerable in this series , maybe because he is the last xxunk . xxmaj the range of emotions which xxmaj chris and xxmaj david show are truly remarkable , and i felt their pain , anger , and sense of adventure right along with them . \n\n xxmaj billie xxmaj piper ( rose ) brings a very human element to each episode , although she is much braver than i would"
2,"xxunk xxunk xxmaj jackson and xxmaj eugene xxmaj xxunk a bad xxunk 's sort of in the vein of "" 48 xxunk i 'm not equating the 2 movies with each other , though i do seem to recall being less than impressed with "" 48 xxunk did however find "" the xxmaj man "" entertaining from beginning to end , from both a dramatic viewpoint and a comedic one.the 2 stars are both very gifted actors , so it","xxunk xxmaj jackson and xxmaj eugene xxmaj xxunk a bad xxunk 's sort of in the vein of "" 48 xxunk i 'm not equating the 2 movies with each other , though i do seem to recall being less than impressed with "" 48 xxunk did however find "" the xxmaj man "" entertaining from beginning to end , from both a dramatic viewpoint and a comedic one.the 2 stars are both very gifted actors , so it was"


#### Fine tune the Language Model 

In order to convert the integer indices to activations for our neural network, we will use embeddings. 

And feed these embeddings into a model architecture. 

The embedding of the pretrained model are merged with the random embeddings added for the words werent in the pretraining vocab. 


In [60]:
learn = language_model_learner(
    dls_lm , AWD_LSTM , drop_mult= .3 , 
    metrics = [accuracy , Perplexity()]
).to_fp16()

Perplexity is the often used metric in the language model, it is the exponential loss (torch.exp(cross_entropy)). 

`language_model_learner` -> will automatically calls the freeze using pretrained model so this will train only on the randomly initialized embedding weights. 



In [61]:

learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.996865,3.896230,0.301687,49.216530,25:51


**Encoder** 

Once the training after unfreezing done, we save all of our model except the final layer that converts activatiosn into probabilities of picking each token in our vocabulary. 


The *encoder* has all the layers of model responsible for the feature extraction except the final layer. 

In Pytorch Dataloaders, we need to collate all the items in a batch into a single tensor has a fixed shape. So we use padding to expand the shortest texts to make them all the same size. 

The special padding token we will be assigning to adjust the lenght will be ignored by the model. 

We do this by approximating the traning set, sorting the documents by length prior to each epoch. We wont pad every batch to the same size instead use the size of the largest document in each batch as the target size. 

The class Numericalize has a decode function which gives us back the string with tokens. 

`numericalizeObject.decode()`

In [65]:
nums_dec = num.decode(nums)

nums_dec

(#30) ['xxunk','years','ago',',','when','xxunk','was','in','high','school'...]

`Tokenizer.decode` turns the above tokens into a single string. 

In [68]:

tkn.decode(nums_dec)

'xxunk years ago , when xxunk was in high school , xxunk was xxunk to read and xxunk xxunk xxunk \'s xxunk xxunk , " xxunk xxunk and xxunk "'

#### Fine tuning the classifier 

For NLP Classifier its found that unfreezing a few layers at a time makes a real difference in the performance of the model. 

`freeze_to` -> -2 value all layers except the last two. 
